In [1]:
# import dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import pandas as pd

In [2]:
#create engine
engine = create_engine('sqlite:///bellybutton.sqlite')


In [3]:
Base = automap_base()

In [4]:
Base.prepare(engine, reflect=True)

In [5]:
Base.classes.keys()

['sample_metadata', 'samples']

In [6]:
Sample_metadata = Base.classes.sample_metadata

In [24]:
Samples =  Base.classes.samples

In [8]:
session = Session(engine)

In [9]:
samples_metadata_ex = session.query(Sample_metadata).first() 
samples_metadata_ex.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x106814a90>,
 'ETHNICITY': 'Caucasian',
 'IMPSURFACE1319': '1',
 'COUNTRY1319': 'usa',
 'GENDER': 'F',
 'NPP1319': None,
 'ZIP1319': '22306',
 'MMAXTEMP1319': 33.99000168,
 'AGE': 24.0,
 'DOG': 'no',
 'WFREQ': 2.0,
 'PFC1319': 25.5,
 'CAT': 'no',
 'BBTYPE': 'I',
 'IMPSURFACE013': 8852.0,
 'LOCATION': 'Beaufort/NC',
 'NPP013': 37.17222214,
 'COUNTRY012': 'usa',
 'sample': 940,
 'MMAXTEMP013': 54.5,
 'ZIP012': '22306',
 'EVENT': 'BellyButtonsScienceOnline',
 'PFC013': None}

In [26]:
stmeta = session.query(Sample_metadata).statement

In [27]:
df_meta = pd.read_sql_query(stmeta, session.bind)
df_meta.head()

,sample,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,None,1,NaN,33.990002,25.5
1,941,None,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,None,None,...,None,None,NaN,NaN,NaN,None,None,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,None,None,...,None,None,NaN,NaN,NaN,None,None,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,None,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,None,0,5613.0,33.634445,24.0


In [11]:
stmt = session.query(Samples).statement
stmt

<sqlalchemy.sql.annotation.AnnotatedSelect at 0x1068b5400; AnnotatedSelect object>

In [28]:
df_meta.WFREQ.dtype


dtype('float64')

In [13]:
# results for an example of 945
results = session.query(Sample_metadata.sample,
        Sample_metadata.ETHNICITY,
        Sample_metadata.GENDER,
        Sample_metadata.AGE,
        Sample_metadata.LOCATION,
        Sample_metadata.BBTYPE,
        ).filter(Sample_metadata.sample == 945).all()
results

[(945, 'Caucasian', 'F', 48.0, 'Philidelphia/PA', 'I')]

In [14]:
sample_dict = {}
for result in results:
    sample_dict["sample"] = result[0]
    sample_dict["ETHNICITY"] = result[1]
    sample_dict["GENDER"] = result[2]
    sample_dict["AGE"] = result[3]
    sample_dict["LOCATION"] = result[4]
    sample_dict["BBTYPE"] = result[5]
    

In [15]:
sample_dict

{'sample': 945,
 'ETHNICITY': 'Caucasian',
 'GENDER': 'F',
 'AGE': 48.0,
 'LOCATION': 'Philidelphia/PA',
 'BBTYPE': 'I'}

In [16]:
resulting = session.query(Sample_metadata.sample,
        Sample_metadata.WFREQ,).filter(Sample_metadata.sample == 940).all()

In [17]:
 wash_freq = {}
    
for result in resulting:
    wash_freq["WFREQ"] = result[1]
wash_freq    

{'WFREQ': 2.0}

In [18]:
df = pd.read_sql_query(stmt, session.bind)
df.head()

,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [30]:

samples = Samples.__table__.columns

In [32]:
samples_list = [sample.key for sample in samples]
samples_list.remove("otu_id")
samples_list.remove("otu_label")
samples_list


['940',
 '941',
 '943',
 '944',
 '945',
 '946',
 '947',
 '948',
 '949',
 '950',
 '952',
 '953',
 '954',
 '955',
 '956',
 '958',
 '959',
 '960',
 '961',
 '962',
 '963',
 '964',
 '966',
 '967',
 '968',
 '969',
 '970',
 '971',
 '972',
 '973',
 '974',
 '975',
 '978',
 '1233',
 '1234',
 '1235',
 '1236',
 '1237',
 '1238',
 '1242',
 '1243',
 '1246',
 '1253',
 '1254',
 '1258',
 '1259',
 '1260',
 '1264',
 '1265',
 '1273',
 '1275',
 '1276',
 '1277',
 '1278',
 '1279',
 '1280',
 '1281',
 '1282',
 '1283',
 '1284',
 '1285',
 '1286',
 '1287',
 '1288',
 '1289',
 '1290',
 '1291',
 '1292',
 '1293',
 '1294',
 '1295',
 '1296',
 '1297',
 '1298',
 '1308',
 '1309',
 '1310',
 '1374',
 '1415',
 '1439',
 '1441',
 '1443',
 '1486',
 '1487',
 '1489',
 '1490',
 '1491',
 '1494',
 '1495',
 '1497',
 '1499',
 '1500',
 '1501',
 '1502',
 '1503',
 '1504',
 '1505',
 '1506',
 '1507',
 '1508',
 '1510',
 '1511',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1517',
 '1518',
 '1519',
 '1521',
 '1524',
 '1526',
 '1527',
 '1530'

In [20]:
otu_descriptions_list = []
for otu_descriptions in df['otu_label']:
    otu_descriptions_list.append(otu_descriptions)
   

In [21]:
# for a sample data of 940-- otu_id and otu_label
result = df.loc[df['940'] > 1, ["otu_id", "otu_label", '940']]
result.head()
    

,otu_id,otu_label,940
40,41,Bacteria,71
120,121,Bacteria,2
158,159,Bacteria,2
164,165,Bacteria,12
169,170,Bacteria,2


In [22]:
#for example sample of 940
data = {
        "otu_ids": result.otu_id.values.tolist(),
        "sample_values": result['940'].values.tolist(),
        "otu_labels": result.otu_label.tolist(),
    }


In [23]:
for key in data.keys():
    for value in data[key]:
        print (key,value)

otu_ids 41
otu_ids 121
otu_ids 159
otu_ids 165
otu_ids 170
otu_ids 189
otu_ids 258
otu_ids 259
otu_ids 307
otu_ids 340
otu_ids 342
otu_ids 352
otu_ids 357
otu_ids 412
otu_ids 482
otu_ids 513
otu_ids 725
otu_ids 830
otu_ids 833
otu_ids 874
otu_ids 907
otu_ids 944
otu_ids 1167
otu_ids 1169
otu_ids 1189
otu_ids 1193
otu_ids 1208
otu_ids 1232
otu_ids 1274
otu_ids 1314
otu_ids 1497
otu_ids 1498
otu_ids 1503
otu_ids 1505
otu_ids 1795
otu_ids 1950
otu_ids 1959
otu_ids 1960
otu_ids 1962
otu_ids 1968
otu_ids 1977
otu_ids 2011
otu_ids 2024
otu_ids 2039
otu_ids 2065
otu_ids 2077
otu_ids 2110
otu_ids 2167
otu_ids 2178
otu_ids 2184
otu_ids 2186
otu_ids 2188
otu_ids 2191
otu_ids 2235
otu_ids 2244
otu_ids 2247
otu_ids 2264
otu_ids 2275
otu_ids 2291
otu_ids 2318
otu_ids 2335
otu_ids 2342
otu_ids 2350
otu_ids 2396
otu_ids 2419
otu_ids 2475
otu_ids 2483
otu_ids 2491
otu_ids 2546
otu_ids 2571
otu_ids 2722
otu_ids 2737
otu_ids 2739
otu_ids 2782
otu_ids 2811
otu_ids 2859
otu_ids 2908
otu_ids 2936
otu_ids 2